### Import the necessary libraries

In [27]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Load data from the csv files

In [28]:
interbankExposure = pd.read_csv('interbankExposures.csv', header=None)
bankEquities = pd.read_csv('bankEquities.csv', header=None)
bank_asset_weighted_network = pd.read_csv('bankAssetWeightedNetwork.csv', header=None)


### Create a directed graph from the dataset

In [29]:
G = nx.DiGraph(interbankExposure.values)

### Calculate the number of nodes in the graph and the in-degree and out-degree of each node

In [30]:
nnodes = G.number_of_nodes()  # 145
out_degrees = [G.out_degree(n) for n in G.nodes()]
in_degrees = [G.in_degree(n) for n in G.nodes()]
print(out_degrees)
print(in_degrees)


[0, 1, 1, 119, 32, 20, 60, 113, 24, 26, 32, 40, 37, 87, 14, 29, 6, 39, 38, 26, 49, 41, 45, 0, 62, 11, 10, 102, 20, 86, 0, 74, 32, 41, 93, 27, 35, 29, 2, 114, 21, 2, 7, 34, 9, 59, 11, 84, 86, 12, 63, 55, 47, 73, 101, 50, 9, 6, 23, 98, 85, 39, 89, 16, 41, 40, 1, 55, 54, 1, 15, 74, 102, 38, 11, 0, 34, 46, 75, 105, 34, 3, 116, 115, 45, 94, 68, 33, 32, 2, 32, 14, 47, 52, 18, 50, 31, 34, 88, 24, 38, 40, 36, 23, 55, 45, 18, 0, 40, 71, 25, 56, 29, 25, 41, 20, 97, 0, 12, 12, 104, 22, 110, 22, 35, 53, 9, 46, 120, 58, 21, 68, 37, 17, 7, 8, 57, 13, 58, 1, 95, 106, 23, 23, 75]
[16, 1, 4, 98, 13, 28, 77, 101, 22, 19, 19, 14, 27, 80, 28, 26, 2, 34, 40, 13, 78, 51, 20, 2, 32, 36, 31, 114, 26, 63, 11, 97, 36, 27, 84, 4, 43, 48, 1, 100, 29, 5, 10, 58, 15, 60, 38, 65, 76, 3, 38, 32, 18, 26, 95, 21, 12, 12, 35, 106, 91, 54, 78, 29, 15, 52, 29, 60, 42, 71, 16, 63, 107, 44, 10, 11, 48, 13, 79, 88, 46, 8, 125, 94, 35, 95, 82, 41, 55, 16, 12, 35, 33, 73, 11, 41, 43, 40, 84, 44, 46, 26, 51, 30, 30, 14, 36, 5, 

### Compute the in degrees for each node (Interbank assets)

In [31]:
in_Degree = pd.DataFrame(G.in_degree(weight='weight'))
in_Degree = in_Degree.T


In [32]:
print(in_Degree.loc[1])

0        538218.69
1          7351.70
2         39050.70
3      85674483.00
4        460659.54
          ...     
140    75172808.00
141    71611906.00
142      303785.69
143     3836714.30
144    21025428.60
Name: 1, Length: 145, dtype: float64


### Compute the out degrees for each node (Interbank Liabilities)

In [33]:
out_Degree = pd.DataFrame(G.out_degree(weight='weight'))
out_Degree = out_Degree.T


In [34]:
print(out_Degree.loc[1])

0      0.000000e+00
1      2.302500e+04
2      2.353800e+04
3      1.771697e+08
4      1.084775e+06
           ...     
140    4.305915e+07
141    1.015331e+08
142    5.896638e+05
143    7.209214e+05
144    2.440614e+07
Name: 1, Length: 145, dtype: float64


In [35]:
print(bankEquities.loc[0])

0        465710.0
1          4436.7
2         13159.0
3      16229000.0
4        438420.0
          ...    
140    11382000.0
141    14033000.0
142      129430.0
143      874880.0
144    26855000.0
Name: 0, Length: 145, dtype: float64


### Calculate the external liabilties for each bank

In [36]:
# Create a new dataframe to store the sum of bank asset weighted network for each bank
bank_asset_weighted_network_sum = pd.DataFrame(index=range(1), columns=range(145))

#store the sum of each row in the bank_asset_weighted_network_sum dataframe
for i in range(145):
    bank_asset_weighted_network_sum.loc[0,i] = bank_asset_weighted_network.loc[i].sum()
print(bank_asset_weighted_network_sum.loc[0])

0        2154600.0
1          29473.2
2         156460.0
3      343060000.0
4        1846500.0
          ...     
140    301110000.0
141    286542000.0
142      1215600.0
143     15375200.0
144     84110000.0
Name: 0, Length: 145, dtype: object


In [37]:
# External_liability = in degree - out_degree - bankEquities + bank_asset_weighted_network

external_liability = pd.DataFrame(index=range(1), columns=range(145))
external_liability = external_liability.sub(bankEquities, fill_value=0)
external_liability = pd.DataFrame(external_liability.loc[0] + in_Degree.loc[1])
external_liability = pd.DataFrame(external_liability[0] - out_Degree.loc[1])
external_liability = pd.DataFrame(external_liability[0] + bank_asset_weighted_network_sum.loc[0])
external_liability = external_liability.T

In [38]:
#print the external liability of each bank
print(external_liability)


          0       1         2            3          4           5    \
0  2227108.69  9363.2  158813.7  235335740.5  783964.75  5038396.01   

          6            7          8           9    ...        135  \
0  63046713.8  309464391.0  2691916.1  1461887.76  ...  447075.82   

           136        137         138       139          140          141  \
0  409762299.9  402529.62 -4821777.76  511817.8  321841655.7  242587818.0   

         142          143         144  
0  800291.94  17616112.88  53874287.9  

[1 rows x 145 columns]


### Separate banks that are not defaulted and defaulted based on their external liabilities:

In [39]:
notdefaulted_external_liability = external_liability[external_liability[:] > 0].dropna(axis=1)
defaulted_external_liability = external_liability[external_liability[:] <= 0].dropna(axis=1)


In [40]:
print(defaulted_external_liability)

        10         11         22         24         35        49          52   \
0 -34248.07 -2847008.5 -2357571.7 -3113241.5 -1254726.8 -152828.5 -2847489.83   

           53          64         77         90         92          98   \
0 -20651114.12 -3169533.98 -4997230.3 -767637.95 -1721770.7 -44793083.3   

         105        114        125        127         138  
0 -4015387.3 -1514734.5 -6645151.5 -1494865.7 -4821777.76  


### Apply the shocks

In [41]:
initial_bank_equities = bankEquities.copy() # Save the initial bank equities to calculate the losses later
shock = 10000000


for i in bankEquities:
    bankEquities[i] = bankEquities[i] - shock

### Separate non-defaulters and defaulters based on their equities:

In [42]:
notdefaulters = bankEquities[bankEquities.iloc[0:] > 0]
notdefaulters = notdefaulters.dropna(axis=1)
defaulters = bankEquities[bankEquities.iloc[0:] <= 0]
defaulters = defaulters.dropna(axis=1)

### ignore below cell

Separate the bank assets of non-defaulted and defaulted banks and calculate the ratio of the sum of the bank assets of non-defaulted banks to the sum of all banks' external assets. This ratio represents the fraction of the total bank assets that remain in the system after accounting for the defaulted banks.

In [43]:
notdefaulted_external_asset = bank_asset_weighted_network.reindex(notdefaulters.index, axis=1)
defaulter_external_assets = bank_asset_weighted_network.reindex(defaulters.index, axis=1)
ratio = 1 - (defaulter_external_assets.sum(axis=1) / bank_asset_weighted_network.sum(axis=1))
print(ratio)

0      1.000000
1      0.833333
2      1.000000
3      1.000000
4      1.000000
         ...   
140    1.000000
141    1.000000
142    0.750000
143    0.750000
144    1.000000
Length: 145, dtype: float64


### Initialize new dataframes for intermediate calculations and updated values in the Furfine function.

In [44]:
newBankEquity = pd.DataFrame(index=range(1), columns=range(145))
newbanklist = pd.DataFrame(index=range(1), columns=range(145))
defaulteroutDegree = pd.DataFrame(index=range(1), columns=range(145))
notdefaulted_external_asset = pd.DataFrame(index=range(1), columns=range(145))

### Subtract non-defaulted external liabilities, and add non-defaulted external assets

In [45]:
newBankEquity = newBankEquity.sub(notdefaulted_external_liability, fill_value=0)
newBankEquity = newBankEquity.add(notdefaulted_external_asset, fill_value=0)

In [46]:
type(bank_asset_weighted_network)

pandas.core.frame.DataFrame

### Define the furfine function

In [47]:
def furfine(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, out_Degree, bank_asset_weighted_network, recovery_rate):
    notdefaulters_inDegree = in_Degree.reindex(notdefaulters.columns, axis=1)
    notdefaulters_outDegree = out_Degree.reindex(notdefaulters.columns, axis=1)

    for i in defaulters:
        sumDebt = sum(interbankExposure.loc[i, 0:])
        recovered = sumDebt * recovery_rate
        defaulted_bank_exposures = interbankExposure.iloc[i, :]
        interbankExposure.iloc[i] = defaulted_bank_exposures * (1 - recovery_rate)
        defaulteroutDegree.at[i, 0] = sumDebt - recovered
        
        defaulted_bank_assets = bank_asset_weighted_network.loc[i, :]
        bank_asset_weighted_network.loc[i, :] = defaulted_bank_assets * (1 - recovery_rate)

    newBankEquity = newBankEquity.sub(defaulteroutDegree, fill_value=0)
    newBankEquity = pd.DataFrame(
        notdefaulters_inDegree.loc[1] + newBankEquity.loc[0])
    newBankEquity = pd.DataFrame(
        newBankEquity[0] - notdefaulters_outDegree.loc[1]).T
    newbanklist = newBankEquity[newBankEquity[:] > 0].dropna(axis=1)

    lenDefaulter = len(newBankEquity[newBankEquity[:] <= 0].dropna(axis=1))
    if len(defaulters) == lenDefaulter:
        return notdefaulters
    else:
        newBankEquity = newbanklist
        defaulters = newBankEquity[newBankEquity[:] <= 0].dropna(axis=0)
        notdefaulters = newBankEquity[newBankEquity[:] > 0].dropna(axis=0)
        return furfine(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, out_Degree, bank_asset_weighted_network)


### Define a recovery rate

In [48]:
recovery_rate = 0.5  # 0.5 = 50% recovery rate

### Call the furfine function

In [49]:

x = furfine(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, out_Degree, bank_asset_weighted_network, recovery_rate)

print(x)

#print the number of banks that have not defaulted
print("The number of banks that have not defaulted are: ")
print(len(x.columns))

         3          7           27         34          39          48   \
0  6229000.0  4267000.0  37698000.0  2562000.0  53616000.0  47097000.0   

        53          54          59        60   ...        83          85   \
0  652000.0  24554000.0  53308000.0  804000.0  ...  8116000.0  18605000.0   

           98         116         122         128         136        140  \
0  158110000.0  7556000.0  36551000.0  17495000.0  22123000.0  1382000.0   

         141         144  
0  4033000.0  16855000.0  

[1 rows x 24 columns]
The number of banks that have not defaulted are: 
24


### Caculate the losses

In [50]:

final_bank_equities = pd.DataFrame(index=range(1), columns=range(145))

final_bank_equities.update(x)
final_bank_equities.fillna(0, inplace=True)


print("Initial Bank Equities: ")
print(initial_bank_equities)
print("Final Bank Equities: ")
print(final_bank_equities)

Initial Bank Equities: 
        0       1      2           3         4         5          6    \
0  465710.0  4436.7  13159  16229000.0  438420.0  271300.0  7572800.0   

          7      8         9    ...    135         136       137       138  \
0  14267000.0  99580  178320.0  ...  32812  32123000.0  699090.0  515310.0   

     139         140         141       142       143         144  
0  14259  11382000.0  14033000.0  129430.0  874880.0  26855000.0  

[1 rows x 145 columns]
Final Bank Equities: 
   0    1    2          3    4    5    6          7    8    9    ...  135  \
0    0    0    0  6229000.0    0    0    0  4267000.0    0    0  ...    0   

          136  137  138  139        140        141  142  143         144  
0  22123000.0    0    0    0  1382000.0  4033000.0    0    0  16855000.0  

[1 rows x 145 columns]


In [51]:
#Compute number of defaults
# defaults are the banks with final equity values <= 0
defaults = final_bank_equities[final_bank_equities.iloc[0:] <= 0].dropna(axis=1)
num_defaults = len(defaults.columns)
print("Number of Defaults:", num_defaults)
    
#Compute contagion effects on other banks
non_defaults = final_bank_equities[final_bank_equities.iloc[0:] > 0].dropna(axis=1)
pct_change = (non_defaults.sum() - initial_bank_equities[initial_bank_equities > 0].sum()) / initial_bank_equities[initial_bank_equities > 0].sum()
print("Percentage Change in Equity Values for Non-Defaults:")
print(pct_change[pct_change.dropna().index])

Number of Defaults: 121
Percentage Change in Equity Values for Non-Defaults:
3     -0.616181
7     -0.700918
27    -0.209652
34    -0.796052
39    -0.157193
48    -0.175141
53    -0.938791
54    -0.289402
59    -0.157958
60    -0.925583
72    -0.564111
78    -0.585343
79    -0.970309
82    -0.091617
83    -0.551998
85    -0.349589
98    -0.059485
116   -0.569606
122   -0.214818
128   -0.363702
136   -0.311303
140   -0.878580
141   -0.712606
144   -0.372370
dtype: float64


### For task 3, we simultaneously consider overlapping portfolios along with counterparty defaults. 

In [62]:
# Helper function to calculate the fraction of asset i owned by banks that have defaulted
def calculate_qi(asset_ownership, defaulted_banks):
    total_owned_by_defaulted_banks = asset_ownership.loc[asset_ownership.index.isin(defaulted_banks)].sum(axis=0)
    total_owned_by_all_banks = asset_ownership.sum(axis=0)
    qi = total_owned_by_defaulted_banks / total_owned_by_all_banks
    return qi

# Helper function to calculate the linear devaluation of assets
def linear_devaluation(asset_prices, qi, a):
    new_asset_prices = asset_prices * (1 - a * qi)
    return new_asset_prices


# Modify the Furfine algorithm function to include the effect of overlapping portfolios
def furfine_with_overlapping_portfolios(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, out_Degree, recovery_rate, bank_asset_weighted_network, a):
    # Calculate the fraction of asset i owned by banks that have defaulted
    qi = calculate_qi(bank_asset_weighted_network, defaulters.columns)
    
    # Update the asset prices based on the linear devaluation function
    asset_prices = linear_devaluation(np.ones(len(qi)), qi, a)
    
    # Update the bank's asset values based on the new asset prices
    updated_bank_asset_values = bank_asset_weighted_network.mul(asset_prices, axis=0)
    
    # Replace the existing Furfine algorithm implementation with the new one that incorporates the effect of overlapping portfolios
    return furfine(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, out_Degree, updated_bank_asset_values, recovery_rate)


# Define the market impact parameter a
a = 0.5  

# Perform stress tests using the modified Furfine algorithm with the same initial shock scenarios as in Task 2
x_task3 = furfine_with_overlapping_portfolios(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, out_Degree, recovery_rate, bank_asset_weighted_network, a)

# Compare the results of these stress tests with those obtained using the Furfine algorithm in Task 2
print("Task 2 results:")
print(x)
print("Task 3 results:")
print(x_task3)

# Compute the differences between the results of Task 2 and Task 3
equity_difference = x_task3.subtract(x, fill_value=0)
print("Equity differences between Task 2 and Task 3:")
print(equity_difference)


Task 2 results:
         3          7           27         34          39          48   \
0  6229000.0  4267000.0  37698000.0  2562000.0  53616000.0  47097000.0   

        53          54          59        60   ...        83          85   \
0  652000.0  24554000.0  53308000.0  804000.0  ...  8116000.0  18605000.0   

           98         116         122         128         136        140  \
0  158110000.0  7556000.0  36551000.0  17495000.0  22123000.0  1382000.0   

         141         144  
0  4033000.0  16855000.0  

[1 rows x 24 columns]
Task 3 results:
         3          7           27         34          39          48   \
0  6229000.0  4267000.0  37698000.0  2562000.0  53616000.0  47097000.0   

        53          54          59        60   ...        83          85   \
0  652000.0  24554000.0  53308000.0  804000.0  ...  8116000.0  18605000.0   

           98         116         122         128         136        140  \
0  158110000.0  7556000.0  36551000.0  17495000.0  221